In [1]:
from pip._internal import main
main(["install","progressbar"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12075 sha256=61934fdd58ca0c9e8c3dadccecaee07d2af6cd6daa949e242d8db7104173fd55
  Stored in directory: /Users/snigdhakakkar/Library/Caches/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar


0

### Importing libraries

In [7]:
import progressbar
import time
import os
import struct
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import numpy as np
import librosa # for sound processing.
import DataCollection as dc # a local module

### Getting metadata

In [8]:
data = pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [9]:
data.shape[0]

8732

In [10]:
dataset = np.zeros(shape = (data.shape[0],2),dtype = object)
dataset.shape

(8732, 2)

### Extracting features

In [11]:
bar = progressbar.ProgressBar(maxval=data.shape[0], widgets=[progressbar.Bar('$', '||', '||'), ' ', progressbar.Percentage()])
bar.start()
for i in range(data.shape[0]):
    
    fullpath, class_id = dc.path_class(data,data.slice_file_name[i])
    try:
        X, sample_rate = librosa.load(fullpath, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    except Exception:
        print("Error encountered while parsing file: ", file)
        mfccs,class_id = None, None
    feature = mfccs
    label = class_id
    dataset[i,0],dataset[i,1] = feature,label
    
    bar.update(i+1)

/Users/snigdhakakkar/opt/anaconda2/envs/tf/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1323                                          ||  40%
  return f(*args, **kwargs)
/Users/snigdhakakkar/opt/anaconda2/envs/tf/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1103
  return f(*args, **kwargs)
/Users/snigdhakakkar/opt/anaconda2/envs/tf/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1523
  return f(*args, **kwargs)


In [12]:
np.save("dataset_mfcc",dataset,allow_pickle=True)

In [14]:
l = np.load("dataset_mfcc.npy",allow_pickle=True)

In [15]:
l.shape

(8732, 2)

In [19]:
l[8700,1]

'drilling'